In [1]:
import cv2
import numpy as np
import mediapipe as mp
import math
import time

In [2]:
# Calculate distance between two points
def calculate_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

In [3]:
# initialize mediapipe
# mphands for controlling the gestures.
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

default_landmark_spec = mpDraw.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)  # Red color
default_connection_spec = mpDraw.DrawingSpec(color=(255, 255, 255), thickness=2)  # White color

In [4]:
def is_click_gesture(landmarks):
    """Detects the CLICK gesture."""
    thumb_tip = landmarks[4]
    index_tip = landmarks[8]
    distance = calculate_distance(thumb_tip, index_tip)
    return distance < 20  # Adjust threshold as needed


In [5]:
def is_scroll_gesture(landmarks):
    """Detects scrolling when index (8) and middle finger (12) are aligned."""
    index_tip = landmarks[8]  # Tip of the index finger
    middle_tip = landmarks[12]  # Tip of the middle finger
    ring_pip = landmarks[14]
    thumb_tip = landmarks[4]

    # Check alignment (both horizontal and vertical alignment with a small tolerance)
    # is_aligned_vertically = abs(index_tip[0] - middle_tip[0]) < 10  # x-coordinates aligned
    is_aligned_horizontally = abs(index_tip[1] - middle_tip[1]) < 20  # y-coordinates aligned
    is_ring_with_thumb = calculate_distance(ring_pip,thumb_tip) < 20

    print(is_ring_with_thumb)

    # Return True if aligned either vertically or horizontally
    # return is_aligned_vertically or is_aligned_horizontally
    return is_ring_with_thumb and is_aligned_horizontally  


# Detecting the scroll direction
def detect_scroll_direction(landmarks, previous_index_x, previous_index_y):
    """Detects if the user is scrolling and determines the direction, with deviation tolerance."""
    index_tip_x = landmarks[8][0]  # x-coordinate of the index finger tip
    index_tip_y = landmarks[8][1]  # y-coordinate of the index finger tip

    # If this is the first frame, initialize the previous position
    if previous_index_x is None or previous_index_y is None:
        return "none", index_tip_x, index_tip_y

    # Define thresholds
    major_threshold = 20  # Major movement threshold (to determine primary direction)
    minor_threshold = 5   # Minor deviation threshold (to ignore small side movements)

    # Calculate differences
    delta_x = index_tip_x - previous_index_x
    delta_y = index_tip_y - previous_index_y

    # Determine scrolling direction with tolerance
    if abs(delta_y) > major_threshold and abs(delta_x) < minor_threshold:
        # If vertical movement is significant and horizontal movement is within minor deviation
        if delta_y > 0:
            direction = "down"
        else:
            direction = "up"
    elif abs(delta_x) > major_threshold and abs(delta_y) < minor_threshold:
        # If horizontal movement is significant and vertical movement is within minor deviation
        if delta_x > 0:
            direction = "right"
        else:
            direction = "left"
    else:
        # No significant movement or mixed movement
        direction = "none"

    # Update previous positions
    return direction, index_tip_x, index_tip_y

In [6]:
# Zoom-In and Zoom-Out
zoom_factor = 1.0  # Initial zoom level
zoom_increment = 0.1  # Zoom sensitivity

def calculate_angle(pointA, pointB, pointC):
    # Vector AB
    AB = np.array([pointB[0] - pointA[0], pointB[1] - pointA[1]])
    # Vector BC
    BC = np.array([pointC[0] - pointB[0], pointC[1] - pointB[1]])

    # Dot product and magnitudes
    dot_product = np.dot(AB, BC)
    magnitude_AB = np.linalg.norm(AB)
    magnitude_BC = np.linalg.norm(BC)

    # Calculate the angle in radians and then convert to degrees
    if magnitude_AB * magnitude_BC == 0:
        return 0
    angle = math.acos(dot_product / (magnitude_AB * magnitude_BC))
    angle_degrees = math.degrees(angle)

    return angle_degrees


In [7]:
def is_zoom_detected(landmarks):
    # Draw the filled green circle for the index finger tip (after drawing connections)
    index_tip = landmarks[8]  # Index finger tip
    middle_tip =landmarks[12] # middle finger tip
    thumb_tip = landmarks[4]
    ring_pip = landmarks[14]

    index_middle_distance = calculate_distance(middle_tip,index_tip)
    thumb_ring_distance = calculate_distance(thumb_tip,ring_pip)


    if 50 < thumb_ring_distance < 100 and index_middle_distance < 18:
        return True
    return False

def detect_zoom_direction(landmarks,previous_distance):
    index_tip = landmarks[8]  # Index finger tip
    thumb_tip = landmarks[4]  # Thumb tip
    middle_tip =landmarks[12]

    result=""

    # Calculate the distances
    thumb_index_distance = calculate_distance(thumb_tip, index_tip)
    thumb_middle_distance = calculate_distance(thumb_tip, middle_tip)
    
    # Calculate the average distance for zooming
    average_distance = (thumb_index_distance + thumb_middle_distance) / 2
    # Determine zoom gesture
    if previous_distance is not None:
        if average_distance < previous_distance - 10:  # Zoom in
            result = "Zoom Out"
        elif average_distance > previous_distance + 10:  # Zoom out
            result = "Zoom In"

    previous_distance = average_distance
    return result,previous_distance


In [8]:
# Printing Commands on the screen 
def print_command(frame,command_text):
    cv2.putText(frame, command_text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

In [9]:
# Ripple Trail Configuration
trail = []

# Define trail properties
trail_max_length = 10  # Maximum number of points in the trail
trail_color = (0, 255, 0)  # Green color
trail_start_radius = 10  # Initial circle radius

In [10]:
# Open the camera
camera = cv2.VideoCapture(0)
isCapturedFrameSuccessful = True

# previous_index of x and y are stored for scroll direction check.
previous_index_x = None
previous_index_y = None
previous_distance = None


last_gesture_time = time.time()

while isCapturedFrameSuccessful:
    isCapturedFrameSuccessful, frame = camera.read()

    frame = cv2.flip(frame, 1)
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process hand landmarks
    hand_process_result = hands.process(framergb)

    if hand_process_result.multi_hand_landmarks:
        for hand_landmarks in hand_process_result.multi_hand_landmarks:
            landmarks = []
            for idx, lm in enumerate(hand_landmarks.landmark):  # Iterate through each landmark
                x = int(lm.x * frame.shape[1])  # Get x coordinate
                y = int(lm.y * frame.shape[0])  # Get y coordinate
                landmarks.append([x, y])

            # Draw connections first
            mpDraw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS, default_landmark_spec, default_connection_spec)

            # Draw the filled green circle for the index finger tip (after drawing connections)
            index_tip = landmarks[8]  # Index finger tip


            
            cv2.circle(frame, (index_tip[0], index_tip[1]), 5, (0, 255, 0), -1)  # Filled green circle

            # Add the index finger position to the trail
            trail.append(index_tip)

            # Limit the trail length
            if len(trail) > trail_max_length:
                trail.pop(0)

            # Draw the ripple trail
            for i, point in enumerate(trail):
                # Calculate the radius and opacity for each circle
                radius = trail_start_radius - int((i / trail_max_length) * trail_start_radius)
                opacity = int(255 * (1 - (i / trail_max_length)))
                color = (trail_color[0], trail_color[1], trail_color[2], opacity)  # RGBA

                # Draw the fading circle
                cv2.circle(frame, point, radius, trail_color, 1)
            # At first check if any scroll is happening or not, if it happens then check the direction.


            
            if is_click_gesture(landmarks) and time.time() - last_gesture_time > 1:
                print_command(frame, "Click")
                last_gesture_time = time.time()

            elif is_zoom_detected(landmarks) and time.time() - last_gesture_time > 1:
                zoom_direction, previous_distance = detect_zoom_direction(landmarks, previous_distance)
                if(zoom_direction!=""):
                    print_command(frame, "Zomming: " + zoom_direction)
                    last_gesture_time = time.time()
            
            elif is_scroll_gesture(landmarks) and time.time() - last_gesture_time > 1:
                print('scrolliiiinnnngggg')
                direction, previous_index_x, previous_index_y  = detect_scroll_direction(landmarks, previous_index_x, previous_index_y)
                if(direction!= "none"):
                    print_command(frame, "Scrolling: " + direction)
                    last_gesture_time = time.time()
                
            
    # Display the frame
    cv2.imshow("Output", frame)
    if cv2.waitKey(1) == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

C:\Users\Mohammad\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
scrolliiiinnnngggg
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
scrolliiiinnnngggg
True
True
True
True
True
True
True
True
True
Tru